In [189]:
import pandas as pd
import numpy as np
import seaborn as sb

In [190]:
dataset = pd.read_csv("C:/Users/62822/RMCS Data/matches.csv", index_col = 0)

dataset.info()
dataset.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1389 entries, 1 to 42
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          1389 non-null   object 
 1   time          1389 non-null   object 
 2   comp          1389 non-null   object 
 3   round         1389 non-null   object 
 4   day           1389 non-null   object 
 5   venue         1389 non-null   object 
 6   result        1389 non-null   object 
 7   gf            1389 non-null   float64
 8   ga            1389 non-null   float64
 9   opponent      1389 non-null   object 
 10  xg            1389 non-null   float64
 11  xga           1389 non-null   float64
 12  poss          1389 non-null   float64
 13  attendance    693 non-null    float64
 14  captain       1389 non-null   object 
 15  formation     1389 non-null   object 
 16  referee       1389 non-null   object 
 17  match report  1389 non-null   object 
 18  notes         0 non-null      float

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City


# Pre-Processing

In [191]:
# away = 1, home = 0

dataset['venue'] = dataset.venue.apply(lambda x: 1 if x=='Away' else 0)
dataset['result'] = dataset.result.apply(lambda x: 1 if x=='W' else 0  if x == 'D' else 2)

dataset = dataset.drop(columns = ['time', 'comp', 'round', 'notes', 'captain', 'referee', 'match report', 'season', 'date', 'day', 'formation', 'opponent'])

dataset['dist'] = dataset['dist'].fillna(dataset['dist'].mean())
dataset['attendance'] = dataset['attendance'].fillna(dataset['attendance'].mean())

In [192]:
from sklearn import preprocessing
dataset = pd.get_dummies(dataset, columns = ['team'])

In [193]:
# Balanced sample
datasetSample0 = dataset[dataset["result"]==0].sample(300)
datasetSample0
datasetSample1 = dataset[dataset["result"]==1].sample(300)
datasetSample1
datasetSample2 = dataset[dataset["result"]==2].sample(300)
datasetSample2
dataset = pd.concat([datasetSample1, datasetSample0, datasetSample2])

In [194]:
dataset['result'].value_counts()

result
1    300
0    300
2    300
Name: count, dtype: int64

In [195]:
X = dataset.drop(columns=['result'])
y = dataset.result

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# Test size = 0.3, Train size = 0.7

# XGBoost

In [196]:
from xgboost import plot_tree
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
Classifier = XGBClassifier(objective='multi:softmax', 
                            num_class=3,  
                            max_depth = 3,
                            learning_rate = 0.1,
                            subsample = 0.05,
                            colsample_bytree = 0.4,
                            seed=42)
Classifier.fit(X_train, y_train)
# plot_tree(Classifier, rankdir='LR')
# plt.show()

print(Classifier.score(X_train, y_train))
print(Classifier.score(X_test, y_test))

prediction = Classifier.predict(X_test)
predictionTrain = Classifier.predict(X_train)

0.8857142857142857
0.8444444444444444


In [197]:
import numpy as np
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, prediction)*100)
print(accuracy_score(y_train, predictionTrain)*100)

print(recall_score(y_test, prediction, average=None)*100)
print(recall_score(y_train, predictionTrain, average=None)*100)

84.44444444444444
88.57142857142857
[70.27027027 97.59036145 90.78947368]
[74.6031746  94.47004608 94.64285714]


In [198]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

target_names = ['1', '2', '0']

print(classification_report(y_test, prediction, target_names=target_names))
print(classification_report(y_train, predictionTrain, target_names=target_names))

              precision    recall  f1-score   support

           1       0.92      0.70      0.80       111
           2       0.87      0.98      0.92        83
           0       0.75      0.91      0.82        76

    accuracy                           0.84       270
   macro avg       0.85      0.86      0.85       270
weighted avg       0.86      0.84      0.84       270

              precision    recall  f1-score   support

           1       0.89      0.75      0.81       189
           2       0.90      0.94      0.92       217
           0       0.87      0.95      0.91       224

    accuracy                           0.89       630
   macro avg       0.89      0.88      0.88       630
weighted avg       0.89      0.89      0.88       630



# SVM

In [199]:
X = dataset.drop(columns=['result'])
y = dataset.result

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# Test size = 0.3, Train size = 0.7

In [200]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
# parameters = [{'kernel':('linear', 'rbf'), 'C':[1, 10], 'random_state':[26, 42], 'decision_function_shape':('ovo')}]
# base_estimator = SVC(random_state=0)
# svm = GridSearchCV(base, parameters, cv = 10, scoring='accuracy')

svm = SVC(random_state=42, kernel = 'rbf',  cache_size = 500, decision_function_shape = 'ovo', gamma = 'scale', C = 3)

In [201]:
clf = svm.fit(X_train, y_train)

In [202]:
print(svm.score(X_train, y_train))
print(svm.score(X_test, y_test))

0.36666666666666664
0.25925925925925924


In [203]:
prediction = clf.predict(X_test)
predictionTrain = clf.predict(X_train)

In [204]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

print(accuracy_score(y_test, prediction)*100)
print(accuracy_score(y_train, predictionTrain)*100)

print(recall_score(y_test, prediction, average=None)*100)
print(recall_score(y_train, predictionTrain, average=None)*100)

25.925925925925924
36.666666666666664
[ 0.         67.46987952 18.42105263]
[ 0.         67.28110599 37.94642857]


In [205]:
# from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

target_names = ['1', '2', '0']

print(classification_report(y_test, prediction, target_names=target_names))
print(classification_report(y_train, predictionTrain, target_names=target_names))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       111
           2       0.30      0.67      0.42        83
           0       0.16      0.18      0.17        76

    accuracy                           0.26       270
   macro avg       0.16      0.29      0.20       270
weighted avg       0.14      0.26      0.18       270

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       189
           2       0.35      0.67      0.46       217
           0       0.39      0.38      0.38       224

    accuracy                           0.37       630
   macro avg       0.25      0.35      0.28       630
weighted avg       0.26      0.37      0.30       630



C:\Users\62822\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\62822\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\62822\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\62822\anaconda3\Lib\site-packages\sklearn\metr

In [206]:
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Ensemble (Using Logistic Regression as the meta model/second model

In [207]:
X = dataset.drop(columns=['result'])
y = dataset.result

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# Test size = 0.3, Train size = 0.7

In [208]:
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

estimators = [
    ('xgboost', XGBClassifier(objective='multi:softmax', 
                            num_class=3,  
                            max_depth = 3,
                            learning_rate = 0.1,
                            subsample = 0.05,
                            colsample_bytree = 0.4,
                            seed=42)),
    ('svm', SVC(random_state=42, kernel = 'rbf', cache_size = 500, decision_function_shape = 'ovo', gamma = 'scale', C = 3))
]

In [209]:
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)

In [210]:
ensemble = clf.fit(X_train, y_train)

In [211]:
print(ensemble.score(X_train, y_train))
print(ensemble.score(X_test, y_test))

0.9111111111111111
0.8703703703703703


In [212]:
prediction = ensemble.predict(X_test)
predictionTrain = ensemble.predict(X_train)

In [216]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

print(accuracy_score(y_test, prediction)*100)
print(accuracy_score(y_train, predictionTrain)*100)

print(recall_score(y_test, prediction, average=None)*100)
print(recall_score(y_train, predictionTrain, average=None)*100)

87.03703703703704
91.11111111111111
[78.37837838 93.97590361 92.10526316]
[83.06878307 94.93087558 94.19642857]


In [214]:
# from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

target_names = ['1', '2', '0']

print(classification_report(y_test, prediction, target_names=target_names))
print(classification_report(y_train, predictionTrain, target_names=target_names))

              precision    recall  f1-score   support

           1       0.91      0.78      0.84       111
           2       0.90      0.94      0.92        83
           0       0.80      0.92      0.86        76

    accuracy                           0.87       270
   macro avg       0.87      0.88      0.87       270
weighted avg       0.87      0.87      0.87       270

              precision    recall  f1-score   support

           1       0.90      0.83      0.87       189
           2       0.94      0.95      0.94       217
           0       0.89      0.94      0.92       224

    accuracy                           0.91       630
   macro avg       0.91      0.91      0.91       630
weighted avg       0.91      0.91      0.91       630

